In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# read data

# filename = './data/driving_log.csv'
filename = './record/driving_log.csv'
df = pd.read_csv(filename, float_precision='high')
data = df.values

print(df.head())

In [ ]:
images = []
measurements = []
for line in data:
    img_path = line[0]
    image = cv2.imread(img_path)    
#     image = cv2.imread("./data/" + img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    images.append(image)
    
    measurement = float(line[3])
    measurements.append(measurement)
    
num_samples = len(images)

# X_train = np.array(images).reshape(num_samples, images[0].shape[0], images[0].shape[1], 1)
X_train = np.array(images)
y_train = np.array(measurements)

In [ ]:
# data overview

print(X_train.shape)
print(y_train.shape)
print(X_train.dtype)
print(y_train.dtype)

In [ ]:
# # data normalize

# # First we should change type of features from int8 to float32 in order to do normalize
# X_train = X_train.astype(np.float32)

# # Then set flags for feature engineering tasks I am going to do.  This will prevent me from skipping an important step.
# is_features_normal = False

# def normalize_dataset(image_data):
#     """
#     Normalize the image data
#     :param image_data: The image data to be normalized
#     :return: Normalized image data
#     """    
#     # +, -, *, / in np.array will automatically broadcast to every element
#     return (image_data - 128.)/128

# if not is_features_normal:
#     print("Normalizing ...")
#     X_train = normalize_dataset(X_train)
#     is_features_normal = True

# print("Finish Normalization !")

# # Make sure we do Normalization before training the model
# assert is_features_normal, 'You skipped the step to normalize the features'

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers import Conv2D, MaxPooling2D, Dropout

In [ ]:
# This is a regression network

# here we do two preprocessing
# (1) normalize the data
# (2) mean centering the data
input_shape = X_train[0].shape

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape))
model.add(Conv2D(32, kernel_size=(11, 11),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64, kernel_size=(7, 7),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(5, 5),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(5, 5),
                activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(256, kernel_size=(3, 3)))
model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

# print(model.summary())


model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, nb_epoch=5, shuffle=True)

model.save('model.h5')